### Crispr Amplicon

Read alignment
Consensus 
Visualisation
Stats

In [100]:
import sys
import pysam
import edlib
from tabulate import tabulate
from collections import Counter

In [182]:
bam = "/mnt/storage1/projects/research/22070I_1057_Cas9-ONT/Sample_22070LRa005/22070LRa005.bam"
bed = "/mnt/storage1/projects/research/22070I_1057_Cas9-ONT/opsin_region.bed"
ref = "/mnt/storage2/megSAP/data/genomes/GRCh38.fa"
fasta_anchors = "anchors.fasta"

Read target bedfile

In [183]:
n_window = 100
roi = []
with open(bed)as f:
    for line in f:
        l = line.strip().split()
        l[1] = max(0, int(l[1]) - n_window)
        l[2] = int(l[2]) + n_window
        roi.append(l)
f.close()

roi

[['chrX', 154143338, 154295780]]

Read reference genome

In [184]:
f = pysam.FastaFile(ref)
seq_ref = f.fetch(reference = roi[0][0], start = roi[0][1], end = roi[0][2])
print("Ref Length:", len(seq_ref))

Ref Length: 152442


Read anchors

In [185]:
f_anchors = pysam.FastxFile(fasta_anchors)
for anchor in f_anchors:
    print(anchor.name, ":", anchor.sequence)
f.close


anchor_BL-715 : GGCAAGGGGGAGGAGGGAAGCCAACAGCAGGATGT
anchor_CHOP_267 : TGGGGGCTAGTGCTGGCACCACCG
anchor_IDT_855 : TGTGCACATGCAAGTCACAAACATTCCAGAT
anchor_CSPO_586 : GGGGCCTGTGGTTGGTGAAGCCCAAGGCA


<function FastaFile.close>

Read Bamfile

In [186]:
b = pysam.AlignmentFile(bam, "rb")
print("Found", b.count(contig = roi[0][0], start = roi[0][1], end = roi[0][2]), "read alignments")

# Extract from documentation:
# Note that region strings are 1-based, while start and end denote an interval in python coordinates. The region is specified by reference, start and end.
# Todo: Maybe reduce coordinates by 1?
reads = {}
for read in b.fetch(contig = roi[0][0], start = roi[0][1], end = roi[0][2]):
    #print(read.query_name)
    if not read.is_supplementary and not read.is_secondary:
        reads[str(read.query_name)] = {
            'name' : read.query_name, 
            'pos'   : read.get_reference_positions(full_length=True),
            'seq_query' : read.query_sequence,
            'pairs' : read.get_aligned_pairs(),
            'strand' : "-" if read.is_reverse else "+"
        }
b.close()

print("Stored", len(reads), "unique sequences")

Found 289 read alignments
Stored 136 unique sequences


Forward/Reverse sequences:

- `read.querysequence` gives the sequence as stored in the BAMfile, always forward
- `read.get_forward_sequence` gives the original read sequence, i.e reverse complement of the BAM-Seq when on reverse strand

When using the forward `querysequence` we get more matches on the anchor sequences on the reverse strand. Probably they wey defined on the forward strand

Test edlib: Align single read to reference

In [188]:

testread =  next(iter(reads.values()))['seq_query']
print("QRY:", testread)
print("REF:", seq_ref)

QRY: TTATTCACTGTGCTGTGCAGCCATCACCACCATCCAGCCGCAGAACTTGTTCATCTTCCCAAACTGAAGCTCTGGCCCTGTTCAACCCTTAACTCCCACTTCAGCCCCTGGCACCCAATTCTTCCACTGTCTGTCTCTAAGGATTTGACCACTCTAGGCCCTTCATATAAGTGGAATTTACAGTATTTGCCTTCCTGTGACTGGCTCATTTTACTGAGCAATGTCCTCAAGGGTCATTCATGTTGGAGCATGTCAAACTTCATTTCTTTATGGGTGACCTCATATTCCATCGTCTGTATAGACCACAATTTGTGTAGCTAATTCAGTCCGTTGATGTTTGTTGTTGATTTGTTTGTTTATTTTGAGACAGTCTTGCCATCAGAGCTCACTGCAGGCTCAACCTCCCAGGCTCAAGAAATCCTCCCGCTATACCTCCCAAGTAGCTGTGACTACAGGCACCTGACTGTGCCTCTGCTAATTTTTGTATTTTGTAGAGACAGGTCCCACTATGCTGCCCAGGCTAGTCTCGAACTCCTGGGCTCAAGTGATCCTCTCACCTCGCCTCCCAAAGAACTGGGATTACAGGCATGAGCTACTACAACCAGCCCAATTGCCCATTGATGGGCACGGTTGCTTCTGTACCAGCTAAGTTGTGAATCACGCTGCTGTGAACATGCGTAGTAAACAGCCCTTCCAGACCTCAGCCTCCATTCCTCTGGGCCTATACCCAGTAGTGCGGTTGCTGGGTCCTATGGGAATTCTATCGCTTAACTTTGGAGGAGCTACAAACTGTTTTCTCTGTTATGGCTGCGCCTATCACAATCCAATTTTAGGATTACTTTATCACCCATAAAGCACTCCCTGTACCCATACAGTCATCCTCCATTTCCCTCCCTCCTGTCCTGGCACCCATTCCTCTGCTTTGTGTGTCTCTGGATTGCCCTATCTAAGCATTTCACAGAGATGGAGCCATGCGCCCGTGAATCTTAGTCTGG

In [189]:
aln = edlib.align(testread, seq_ref, task="path", mode = "HW")
print(aln)
print(edlib.getNiceAlignment(aln, testread, seq_ref))

{'editDistance': 845, 'alphabetLength': 4, 'locations': [(104351, 105909), (104351, 105914), (104351, 105915)], 'cigar': '2=1I3=2I1=1I1=2I1=3X1=1D5=1D2=1I1=1I2=1X1=1X2=2I6=1D2=2D1=1X3=2X1D3=5I1=2I2=2X1=1I1=1X1=3I2=1I3=1X1I1=1I2=1I4=1I1=1X1=1I1=2D1=1X1I7=1I2=1X1=1X1=3I3=2X1=1D1=1X2=1I2=1X2=1I3=1X1I2=1X2=1X1=1I1=2X1I1=1I1=1I1=1I1=1X1=2X1=1I1=1X1I1=2I2=1I1=2X1=2I2=2I2=2X2=1I1=1X4=3I3=1X3=1X4I1=2I2=1I1=1X1I1=2X4=1X1I1=2X2=1X2I3=2X1I1=1X1=1I2=2X1=1I2=1I1=1X2=1I1=1D3=1X2I2=1X2I3=1X1I5=2I1=1I2=1X1I3=1X1=1X1=9X3=1X2=1I1=1I1=1I1=3I3=3I1=2I1=1I1=1X2=1I1=1X1=1X1=1I1=1I1=6X1=1X2I2=3X1=1X1=1I4=2X3=1X4=1D2=1I10=1X11=1X8=2X4=4D7=1D1=1X1I5=2X21=1D1=1I1=1X1=1D1=2X1D3=1I1=1X10=1D1=1D1=1X4=1D5=1X2=1X2=1D2=1X7=1X10=1X5=1X8=1X7=1X1D1=1I2=1I1=1D1=1X6=1X1D10=2X2=1X11=1X5=2X1=1X2=2X8=1X2=2D2=3X2=1X3=2I2=2X1I2=1I1=1X4=2I2=2X2=1X1=1X1=1I3=2X2=1I1=1I1=2I3=2I1=1I2=2I1=2I1=1X1I1=2I2=1X2=3X2=1D1=1I2=1X1=1X2I2=1I1=1I1=4X2=1X3=1X1I1=1X1=1I2=1X2=1X1=1X1D2=1I3=1D1=1I2=1X2=1I1=1I5=2X1=1X3=2I1=4I1=1X1I1=4I2=1I2=2I3=3I2=4

Test edlib: Align single read to anchor sequence


In [190]:
f_anchors = pysam.FastxFile(fasta_anchors)
anchor1 = next(f_anchors).sequence
f.close()

# Use HW (Infix) mode to find the anchor sequenc
aln = edlib.align(anchor1, testread, task="path", mode =  "HW")
print(aln)
print(edlib.getNiceAlignment(aln, anchor1, testread))


{'editDistance': 12, 'alphabetLength': 4, 'locations': [(1622, 1654)], 'cigar': '2=1I1=1I1=2X4=2D2=1X5=1X1=1D2=1I2=2I6='}
{'query_aligned': 'GGCAAGGGGGAG--GAGGGAAGCC-AACAGCAGGATGT', 'matched_aligned': '||-|-|..||||--||.|||||.|-||-||--||||||', 'target_aligned': 'GG-A-GCTGGAGCTGATGGAAGTCGAA-AG--GGATGT'}


Align all anchors to all reads

In [191]:
anchor_alignments = {}
for read in reads:
    seq = reads[read]['seq_query']
    anchor_alignments[read] = {}
    for anchor in pysam.FastxFile(fasta_anchors):
        aln = edlib.align(anchor.sequence, seq, task="path", mode="HW")
        anchor_alignments[read][anchor.name] = aln

print("Length of anchor table:", len(anchor_alignments))

Length of anchor table: 136


Count anchor matches (Treshold set at 5)

In [192]:
DISTANCE_THRESHOLD= 5
matching_anchors = []
matching_reads = []
for read in anchor_alignments:
    for anchor in anchor_alignments[read]:
        if anchor_alignments[read][anchor]['editDistance'] < DISTANCE_THRESHOLD:
            matching_anchors.append(anchor)
            matching_reads.append(read)

print(Counter(matching_anchors))        # Number of reads per anchor
print(len(Counter(matching_reads)))     # Number of reads with at least 1 anchor
print(Counter(matching_reads))          # Number of anchors per read


Counter({'anchor_CHOP_267': 42, 'anchor_IDT_855': 32, 'anchor_BL-715': 31, 'anchor_CSPO_586': 25})
91
Counter({'2c28252f-5155-43d0-9f36-6532b7c256cc': 2, 'ad78d953-0fb5-4faf-979d-4afe0935a43a': 2, '90ef129f-51a4-472e-886e-a36a7a65aeed': 2, '1e3e71a9-297f-4c80-9312-5a21d643def7': 2, 'a14ed15b-13c9-4bc7-b75e-af70eefea4d5': 2, 'eb89831e-e29b-43de-be56-6035fd33e48e': 2, '7a49d01b-066c-4c73-bf78-4796501bc46b': 2, '41bcd08f-4ae5-4581-b969-2ef8a4fe69b7': 2, 'ed251a40-838d-491f-b6c7-197fcb5ccd36': 2, 'd7cb37e0-1ef0-4e19-a739-95fd0ca2ccb5': 2, '8a64d7d2-c3e9-4884-9082-81b2094c2834': 2, 'dc72fcac-c658-4490-b309-4b3d3d5ffdab': 2, '7c8b0f69-da8d-4e0c-8158-b44ba2aaf76a': 2, '30fceddb-c645-47fb-8a3c-06e088ea26be': 2, '9bd68884-ab3a-4fa2-b320-3feef4dc0dd6': 2, '302e8a1a-0c4f-4705-9295-1830e67dae95': 2, 'c27a5b4a-9c9a-4963-9ff2-edd5465f5032': 2, 'e093a7c4-5be6-4de8-979b-e7e54884eb63': 2, '2ce8c32d-adb5-41d3-9e75-f41e08a9347b': 2, 'bb095ebd-96f3-403f-a9db-87d6b3c201f1': 2, '99336c8b-19c8-40b2-9a0f-2764

Reads that match more then one anchor sequence span multiple anchors. To get the correct start position we need to chose the correct anchor sequence as follows: 
 - Take  the anchor with _lowest_ coordinates when on the forward strand
 - Select the anchor with _highest_ coordinates when on the reverse strand

In order to get correct coordinates: 

1) First align anchors to ref genome
2) Extract coordinates
3) Sort anchors matches ascending start
4) Sort anchors matches by descending end

In [206]:
anchors = {}
for anchor in pysam.FastxFile(fasta_anchors):
    
    aln = edlib.align(anchor.sequence, seq_ref, task = "path", mode = "HW")
    if  aln['editDistance'] > 0:
        print("WARNING: no perfect match for ", anchor.name) 
        print("Best anchor match:", aln)

    anchors[anchor.name] = {
        'start': aln['locations'][0][0],
        'end' : aln['locations'][0][1] 
        }

print(anchors)

anchors_sorted_forward = sorted(anchors.items(), key=lambda x:x[1]['start'])
anchors_sorted_reverse = sorted(anchors.items(), key=lambda x:x[1]['end'], reverse=True)

print(anchors_sorted_start)
print(anchors_sorted_reverse)

{'anchor_BL-715': {'start': 322, 'end': 356}, 'anchor_CHOP_267': {'start': 151726, 'end': 151749}, 'anchor_IDT_855': {'start': 152305, 'end': 152335}, 'anchor_CSPO_586': {'start': 189, 'end': 217}}
[('anchor_CSPO_586', {'start': 189, 'end': 217}), ('anchor_BL-715', {'start': 322, 'end': 356}), ('anchor_CHOP_267', {'start': 151726, 'end': 151749}), ('anchor_IDT_855', {'start': 152305, 'end': 152335})]
[('anchor_IDT_855', {'start': 152305, 'end': 152335}), ('anchor_CHOP_267', {'start': 151726, 'end': 151749}), ('anchor_BL-715', {'start': 322, 'end': 356}), ('anchor_CSPO_586', {'start': 189, 'end': 217})]


In [175]:
DISTANCE_THRESHOLD= 5
matching_anchors = []
matching_reads = []
for read in anchor_alignments:
    for anchor in anchor_alignments[read]:
        if anchor_alignments[read][anchor]['editDistance'] < DISTANCE_THRESHOLD:
            
            matching_reads.append(read)

['anchor_BL-715',
 'anchor_CSPO_586',
 'anchor_CSPO_586',
 'anchor_CSPO_586',
 'anchor_BL-715',
 'anchor_CSPO_586',
 'anchor_BL-715',
 'anchor_CSPO_586',
 'anchor_BL-715',
 'anchor_CSPO_586',
 'anchor_CSPO_586',
 'anchor_CSPO_586',
 'anchor_CSPO_586',
 'anchor_BL-715',
 'anchor_CSPO_586',
 'anchor_BL-715',
 'anchor_CSPO_586',
 'anchor_BL-715',
 'anchor_CSPO_586',
 'anchor_CSPO_586',
 'anchor_BL-715',
 'anchor_CSPO_586',
 'anchor_BL-715',
 'anchor_CSPO_586',
 'anchor_CSPO_586',
 'anchor_BL-715',
 'anchor_CSPO_586',
 'anchor_BL-715',
 'anchor_CSPO_586',
 'anchor_BL-715',
 'anchor_CSPO_586',
 'anchor_BL-715',
 'anchor_CSPO_586',
 'anchor_CSPO_586',
 'anchor_BL-715',
 'anchor_BL-715',
 'anchor_BL-715',
 'anchor_BL-715',
 'anchor_BL-715',
 'anchor_BL-715',
 'anchor_BL-715',
 'anchor_BL-715',
 'anchor_BL-715',
 'anchor_BL-715',
 'anchor_CHOP_267',
 'anchor_BL-715',
 'anchor_CHOP_267',
 'anchor_IDT_855',
 'anchor_BL-715',
 'anchor_IDT_855',
 'anchor_CHOP_267',
 'anchor_CHOP_267',
 'anchor_IDT

Extract positions after matching anchors for every read. 

In [ ]:
downstream_coords = {}
for read in anchor_alignments:
    for anchor in anchor_alignments:
        if anchor_alignments[read][anchor]['editDistance'] < DISTANCE_THRESHOLD:
            